Recreate experiments

In [3]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5010")

# Trouver l'expérience supprimée
experiments = client.search_experiments(view_type=3)

for exp in experiments:
    if exp.name == "lung-cancer-detection_baseline":
        print(f"Suppression définitive de l'expérience ID: {exp.experiment_id}")
        
        # Supprimer définitivement (⚠️ IRRÉVERSIBLE!)
        client.delete_experiment(exp.experiment_id)
        print(f"✅ Expérience supprimée définitivement!")
        break

# Maintenant vous pouvez créer une nouvelle expérience avec le même nom
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5010")
mlflow.set_experiment("lung-cancer-detection_baseline")
print("✅ Nouvelle expérience créée!")

Suppression définitive de l'expérience ID: 5


RestException: RESOURCE_DOES_NOT_EXIST: No Experiment with id=5 exists

In [2]:
import psycopg2

def clean_experiment_complete(experiment_id):
    """Supprime complètement une expérience et TOUTES ses dépendances"""
    try:
        conn = psycopg2.connect(
            host="localhost",
            port=5432,
            database="mlflow",
            user="mlflow",
            password="mlflow"
        )
        
        cursor = conn.cursor()
        
        # Statistiques
        cursor.execute(f"SELECT COUNT(*) FROM runs WHERE experiment_id = '{experiment_id}';")
        run_count = cursor.fetchone()[0]
        
        cursor.execute(f"SELECT name FROM experiments WHERE experiment_id = '{experiment_id}';")
        exp_name = cursor.fetchone()
        exp_name = exp_name[0] if exp_name else "Inconnue"
        
        print(f"📊 Expérience: {exp_name} (ID: {experiment_id})")
        print(f"📊 Runs à supprimer: {run_count}")
        
        # Liste complète des tables à nettoyer dans l'ordre
        cleanup_queries = [
            # Tables liées aux modèles loggés
            ("logged_model_params", 
             f"DELETE FROM logged_model_params WHERE run_id IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            ("logged_model_tags", 
             f"DELETE FROM logged_model_tags WHERE run_id IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            ("logged_models", 
             f"DELETE FROM logged_models WHERE run_id IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            # Tables liées aux inputs/outputs
            ("inputs", 
             f"DELETE FROM inputs WHERE destination_id IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            ("input_tags", 
             f"DELETE FROM input_tags WHERE input_uuid IN (SELECT input_uuid FROM inputs WHERE destination_id IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}'));"),
            
            # Tables liées aux datasets
            ("dataset_inputs", 
             f"DELETE FROM dataset_inputs WHERE destination_id IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            # Tables liées aux traces
            ("trace_request_metadata", 
             f"DELETE FROM trace_request_metadata WHERE request_id IN (SELECT request_id FROM trace_info WHERE experiment_id = '{experiment_id}');"),
            
            ("trace_tags", 
             f"DELETE FROM trace_tags WHERE request_id IN (SELECT request_id FROM trace_info WHERE experiment_id = '{experiment_id}');"),
            
            ("trace_info", 
             f"DELETE FROM trace_info WHERE experiment_id = '{experiment_id}';"),
            
            # Tables liées aux runs (métriques, params, tags)
            ("latest_metrics", 
             f"DELETE FROM latest_metrics WHERE run_uuid IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            ("metrics", 
             f"DELETE FROM metrics WHERE run_uuid IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            ("params", 
             f"DELETE FROM params WHERE run_uuid IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            ("tags", 
             f"DELETE FROM tags WHERE run_uuid IN (SELECT run_uuid FROM runs WHERE experiment_id = '{experiment_id}');"),
            
            # Les runs eux-mêmes
            ("runs", 
             f"DELETE FROM runs WHERE experiment_id = '{experiment_id}';"),
            
            # Tags de l'expérience
            ("experiment_tags", 
             f"DELETE FROM experiment_tags WHERE experiment_id = '{experiment_id}';"),
            
            # L'expérience elle-même
            ("experiments", 
             f"DELETE FROM experiments WHERE experiment_id = '{experiment_id}';"),
        ]
        
        # Exécuter les requêtes de nettoyage
        for table_name, query in cleanup_queries:
            try:
                print(f"🗑️  Nettoyage de {table_name}...")
                cursor.execute(query)
            except Exception as e:
                # Si la table n'existe pas ou qu'il n'y a rien à supprimer, continuer
                if "does not exist" not in str(e):
                    print(f"   ⚠️  Avertissement pour {table_name}: {e}")
        
        conn.commit()
        print(f"✅ Expérience '{exp_name}' complètement supprimée de la base!")
        
        cursor.close()
        conn.close()
        
    except Exception as e:
        print(f"❌ Erreur finale: {e}")
        if conn:
            conn.rollback()
        raise

# Utilisation
clean_experiment_complete('5')

📊 Expérience: lung-cancer-detection_baseline (ID: 5)
📊 Runs à supprimer: 24
🗑️  Nettoyage de logged_model_params...
🗑️  Nettoyage de logged_model_tags...
   ⚠️  Avertissement pour logged_model_tags: current transaction is aborted, commands ignored until end of transaction block

🗑️  Nettoyage de logged_models...
   ⚠️  Avertissement pour logged_models: current transaction is aborted, commands ignored until end of transaction block

🗑️  Nettoyage de inputs...
   ⚠️  Avertissement pour inputs: current transaction is aborted, commands ignored until end of transaction block

🗑️  Nettoyage de input_tags...
   ⚠️  Avertissement pour input_tags: current transaction is aborted, commands ignored until end of transaction block

🗑️  Nettoyage de dataset_inputs...
   ⚠️  Avertissement pour dataset_inputs: current transaction is aborted, commands ignored until end of transaction block

🗑️  Nettoyage de trace_request_metadata...
   ⚠️  Avertissement pour trace_request_metadata: current transaction i